This notebook is going to mostly scratch work looking at the "sample size" simulation.

Specifically, I'll start with a case-control dataset (to simulate a "real" microbiome effect) and:

- subsample cases and controls at different total N's
- also subsample with different proportions of cases/controls
- calculate significant associations
    - need to remove OTUs which are completely zero before doing this
    - and then correct for multiple tests
- and see what percent of the "real" associations it recovers ("real" associations calculated from the whole dataset)

In this exploration, I also need to make sure that the bugs which are significant in the smaller subset are the same ones as in the whole thing...

In [3]:
import pandas as pd
import feather

from scipy.stats.mstats import kruskalwallis
from scipy.stats import ranksums, mannwhitneyu
# FDR correction
from statsmodels.sandbox.stats.multicomp import multipletests


In [61]:
# Taken from util.py in microbiomeHD
def univariate_one_col(col, xsmpls, ysmpls, pfun):
    try:
        h, p = pfun(col.loc[xsmpls], col.loc[ysmpls])
    except:
        p = 1
        h = 0
    return pd.Series([p, h])

def compare_otus_teststat(df, Xsmpls, Ysmpls, method='kruskal-wallis', multi_comp=None):
    """
    Compares columns between Xsmpls and Ysmpls, with statistical method=method.
    Returns dataframe with both the qvals ('p') and test statistic ('test-stat')

    parameters
    ----------
    df             dataframe, samples are in rows and OTUs in columns
    X,Ysmpls       list of samples to compare
    method         statistical method to use for comparison
    multi_comp     str, type of multiple comparison test to do.
                   Currently accepts 'fdr' or None

    outputs
    -------
    results        dataframe with OTUs in rows and 'p' and 'test-stat' in columns

    """
    if method == 'kruskal-wallis':
        pfun = kruskalwallis
    elif method == 'wilcoxon' or method == 'ranksums':
        pfun = ranksums
    elif method == 'mann-whitney':
        pfun = mannwhitneyu
        # Note: prob wanna add some kwargs here to say whether 2sided or not

#     results = pd.DataFrame(index=df.columns, columns=['test-stat', 'p'])
#     for o in df.columns:
#         try:
#             h, p = pfun(df.loc[Xsmpls, o], df.loc[Ysmpls, o])
#         except:
#             p = 1
#             h = 0
#         results.loc[o, 'p'] = p
#         results.loc[o, 'test-stat'] = h
    results = df.apply(lambda col: univariate_one_col(col, h, crc, pfun)).T
    results.columns = ['p', 'test_stat']

    if multi_comp == 'fdr':
        _, results['q'], _, _ = multipletests(results['p'], method='fdr_bh')

    return results

def read_dataframes(fotu, fmeta):
    df = feather.read_dataframe(fotu)
    df.index = df.iloc[:,0]
    df = df.iloc[:, 1:]
    
    meta = feather.read_dataframe(fmeta)
    meta.index = meta.iloc[:, 0]
    meta = meta.iloc[:, 1:]
    
    return df, meta

# The setup

The eventual plots will be faceted by FMT response rate, x-axis = total N, y-axis = % of max rejections, and each line will be a study.

So I'll need to make a dataframe with the following columns:
- % cases (i.e. FMT response rate)
- total N (x axis)
- number reject (y axis)
- total reject (for that study)
- study (hue)

## CRC Baxter

Let's start with one dataset, CRC Baxter.

In [15]:
fotu = '../../data/clean/crc_baxter.otu_table.feather'
fmeta = '../../data/clean/crc_baxter.metadata.feather'

df, meta = read_dataframes(fotu, fmeta)
df.shape, meta.shape

((490, 18448), (490, 72))

In [20]:
df.head()

,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Peptostreptococcaceae;g__Clostridium_XI;s__;d__denovo15989,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Bacteroides;s__;d__denovo15988,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Ruminococcaceae;g__;s__;d__denovo723,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Ruminococcaceae;g__Faecalibacterium;s__;d__denovo722,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Prevotellaceae;g__Prevotella;s__;d__denovo12511,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Lachnospiraceae;g__Blautia;s__;d__denovo12510,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Lachnospiraceae;g__;s__;d__denovo12516,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Ruminococcaceae;g__;s__;d__denovo20538,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__;g__;s__;d__denovo20532,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Ruminococcaceae;g__Ruminococcus;s__;d__denovo20533,...,k__Bacteria;p__Verrucomicrobia;c__Verrucomicrobiae;o__Verrucomicrobiales;f__Verrucomicrobiaceae;g__Akkermansia;s__;d__denovo14612,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Lachnospiraceae;g__Lachnospiracea_incertae_sedis;s__;d__denovo20807,k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Coriobacteriales;f__Coriobacteriaceae;g__Enterorhabdus;s__;d__denovo19190,k__Bacteria;p__Firmicutes;c__Negativicutes;o__Selenomonadales;f__Acidaminococcaceae;g__Phascolarctobacterium;s__;d__denovo19192,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Lachnospiraceae;g__;s__;d__denovo19193,k__Bacteria;p__Verrucomicrobia;c__Verrucomicrobiae;o__Verrucomicrobiales;f__Verrucomicrobiaceae;g__Akkermansia;s__;d__denovo19195,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Lachnospiraceae;g__Dorea;s__;d__denovo19196,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Ruminococcaceae;g__;s__;d__denovo19197,k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Bifidobacteriales;f__Bifidobacteriaceae;g__Bifidobacterium;s__;d__denovo19198,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Bacteroides;s__;d__denovo20801
index,,,,,,,,,,,,,,,,,,,,,
2045653,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2087650,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2963670,0,0,0,2,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
2527670,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3453650,0,0,0,2,0,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0


In [24]:
# Convert to relative abundance
df = df.divide(df.sum(axis=1), axis=0)

In [25]:
meta.groupby(['DiseaseState']).size()

DiseaseState
CRC       120
H         172
nonCRC    198
dtype: int64

Let's do CRC vs. H

In [62]:


h = meta.query('DiseaseState == "H"').index.tolist()
crc = meta.query('DiseaseState == "CRC"').index.tolist()

p = compare_otus_teststat(df, h, crc, method='kruskal-wallis', multi_comp='fdr')